# Merging Data Scrapes
___

The data has been scraped from Reddit through the API.  The data is coming in the form of 4 files:

* One file for the titles of Subreddit community one
* One file for the titles of Subreddit community two
* One file for the transaction logs of Subreddit community one
* One file for the transaction logs of Subreddit community two

This data can easily be merged into two files, one as a transaction log and the other the posts.

The following cells will accomplish that with some routine cleaning and sorting.
___

In [2]:
import pandas as pd

### Initial read in
___

In [3]:
df1 = pd.read_csv('./nfl_transaction_log.csv')
df2 = pd.read_csv('./nhl_transaction_log.csv')
df3 = pd.read_csv('./nfl_pulls.csv')
df4 = pd.read_csv('./nhl_pulls.csv')

### Transaction logs
___

In [4]:
# How many scrapes amongst transaction logs
dfs = [df1, df2]
scrapes = 0
for df in dfs:
    scrapes += len(df)
scrapes

22

In [5]:
# Combine different subreddit transaction logs
df_tlogs = pd.concat(dfs)
df_tlogs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 0 to 10
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   subreddit        22 non-null     object
 1   pulled_postings  22 non-null     int64 
 2   total_postings   22 non-null     int64 
 3   times_created    22 non-null     object
 4   anchor           22 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.0+ KB


In [6]:
# If you didn't merge between pulls, header may have been written underneath
drop = df_tlogs[df_tlogs['subreddit'] == 'subreddit']
df_tlogs.drop(index=drop.index, inplace=True)
df_tlogs.shape

(22, 5)

In [7]:
# Update merged transactions log
merged_logs = pd.read_csv('./merged_logs.csv')
merged_logs = pd.concat([merged_logs, df_tlogs])
merged_logs.sort_values('times_created', inplace=True)
merged_logs

,subreddit,pulled_postings,total_postings,times_created,anchor
0,nhl,27,27,Fri Oct 11 00:28:57 2024,t3_1fzha42
0,nfl,27,27,Fri Oct 11 00:28:58 2024,t3_1g0z5r2
1,nhl,100,127,Fri Oct 11 00:29:01 2024,t3_1fqbud0
1,nfl,100,127,Fri Oct 11 00:29:06 2024,t3_1fzsf4x
2,nhl,100,227,Fri Oct 11 00:29:11 2024,t3_1fqbud0
...,...,...,...,...,...
815,nfl,100,1100,Wed Oct 9 22:44:50 2024,t3_1fyco23
816,nfl,100,1100,Wed Oct 9 22:44:53 2024,t3_1fyco23
817,nfl,100,1100,Wed Oct 9 22:44:55 2024,t3_1fyco23
818,nfl,100,1100,Wed Oct 9 22:44:58 2024,t3_1fyco23


### Writing Tranaction Log
___

In [8]:
merged_logs.sort_values('times_created', inplace=True)
merged_logs.to_csv('./merged_logs.csv', index=False)

### Gathering Posts from both Subreddit communities
___

In [9]:
df3.info(), df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   time_pulled    127 non-null    object
 1   fullname       127 non-null    object
 2   titles         127 non-null    object
 3   times_created  127 non-null    object
 4   anchors        127 non-null    object
 5   subreddit      127 non-null    object
dtypes: object(6)
memory usage: 6.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   time_pulled    127 non-null    object
 1   fullname       127 non-null    object
 2   titles         127 non-null    object
 3   times_created  127 non-null    object
 4   anchors        127 non-null    object
 5   subreddit      127 non-null    object
dtypes: object(6)
memory usage: 6.1+ KB


(None, None)

In [10]:
# How many were there
dfs_pulls = [df3, df4]
scrapes_pulls = 0
for df in dfs_pulls:
    scrapes_pulls += len(df)
scrapes_pulls

254

In [11]:
# Combine the posts from both subreddits
df_posts = pd.concat(dfs_pulls)
df_posts.sort_values('times_created', inplace=True)
df_posts.shape

(254, 6)

In [12]:
# Drop duplicates, check to see if any
df_posts.sort_values('titles', inplace=True)
df_posts.head(10)

,time_pulled,fullname,titles,times_created,anchors,subreddit
49,Fri Oct 11 00:28:59 2024,t3_1fyc39m,'Tough business': NHLers on the league's volat...,2024-10-07 09:36:40,t3_1fqbud0,nhl
58,Fri Oct 11 00:29:05 2024,t3_1g0k7yv,2024 NFL QB Analysis: Time to Throw vs. Comple...,2024-10-10 07:33:31,t3_1fzsf4x,nfl
81,Fri Oct 11 00:29:00 2024,t3_1fv6dwh,2024-25 NHL Team Twitter (or X) Logo Hashtags,2024-10-03 05:21:50,t3_1fqbud0,nhl
45,Fri Oct 11 00:28:59 2024,t3_1fyjkrh,5 iconic Henrik Lundqvist saves for you 😌,2024-10-07 14:44:37,t3_1fqbud0,nhl
48,Fri Oct 11 00:28:59 2024,t3_1fyenx2,5 waiver claims which one is the biggest W,2024-10-07 11:22:03,t3_1fqbud0,nhl
96,Fri Oct 11 00:29:00 2024,t3_1ftfb3j,7 Trade Landing Spots for Boston Bruins Goalie...,2024-09-30 21:04:44,t3_1fqbud0,nhl
19,Fri Oct 11 00:28:57 2024,t3_1fzvud9,A pic of Amerant Bank Arena from this morning,2024-10-09 09:40:14,t3_1fzha42,nhl
18,Fri Oct 11 00:28:58 2024,t3_1g0w22k,"After the Jets loss to the Vikings, Aaron Rodg...",2024-10-10 16:18:35,t3_1g0z5r2,nfl
81,Fri Oct 11 00:29:05 2024,t3_1g02ixw,Agent's Take: Raiders trading Davante Adams se...,2024-10-09 14:21:37,t3_1fzsf4x,nfl
26,Fri Oct 11 00:28:57 2024,t3_1fkp639,All Streaming Questions and All Broadcast Disc...,2024-09-19 09:33:07,t3_1fzha42,nhl


In [13]:
# Drop actual dupes
df_posts.drop_duplicates(subset=['titles'], keep='first', inplace=True)
df_posts.head(10)

,time_pulled,fullname,titles,times_created,anchors,subreddit
49,Fri Oct 11 00:28:59 2024,t3_1fyc39m,'Tough business': NHLers on the league's volat...,2024-10-07 09:36:40,t3_1fqbud0,nhl
58,Fri Oct 11 00:29:05 2024,t3_1g0k7yv,2024 NFL QB Analysis: Time to Throw vs. Comple...,2024-10-10 07:33:31,t3_1fzsf4x,nfl
81,Fri Oct 11 00:29:00 2024,t3_1fv6dwh,2024-25 NHL Team Twitter (or X) Logo Hashtags,2024-10-03 05:21:50,t3_1fqbud0,nhl
45,Fri Oct 11 00:28:59 2024,t3_1fyjkrh,5 iconic Henrik Lundqvist saves for you 😌,2024-10-07 14:44:37,t3_1fqbud0,nhl
48,Fri Oct 11 00:28:59 2024,t3_1fyenx2,5 waiver claims which one is the biggest W,2024-10-07 11:22:03,t3_1fqbud0,nhl
96,Fri Oct 11 00:29:00 2024,t3_1ftfb3j,7 Trade Landing Spots for Boston Bruins Goalie...,2024-09-30 21:04:44,t3_1fqbud0,nhl
19,Fri Oct 11 00:28:57 2024,t3_1fzvud9,A pic of Amerant Bank Arena from this morning,2024-10-09 09:40:14,t3_1fzha42,nhl
18,Fri Oct 11 00:28:58 2024,t3_1g0w22k,"After the Jets loss to the Vikings, Aaron Rodg...",2024-10-10 16:18:35,t3_1g0z5r2,nfl
81,Fri Oct 11 00:29:05 2024,t3_1g02ixw,Agent's Take: Raiders trading Davante Adams se...,2024-10-09 14:21:37,t3_1fzsf4x,nfl
26,Fri Oct 11 00:28:57 2024,t3_1fkp639,All Streaming Questions and All Broadcast Disc...,2024-09-19 09:33:07,t3_1fzha42,nhl


In [14]:
# Compare size
df_posts.shape

(254, 6)

### Concat with newly scraped posts
____

In [15]:
# Calling in existing posts
merged_pulls = pd.read_csv('./merged_pulls.csv')
merged_pulls.shape

(2422, 6)

In [18]:
merged_pulls = pd.concat([merged_pulls, df_posts])
merged_pulls.sort_values('time_pulled', inplace=True)

In [19]:
# Drop any duplicates
merged_pulls.drop_duplicates(subset=['titles'], keep='first', inplace=True)
merged_pulls.head(10)

,time_pulled,fullname,titles,times_created,anchors,subreddit
87,Fri Oct 4 00:02:54 2024,NaN,Are the Red Wings the most overrated team goin...,Thu Oct 3 22:51:06 2024,t3_1ftfb3j,nhl
429,Fri Oct 4 00:02:54 2024,NaN,Great job EA (NHL 25 threes),Thu Oct 3 23:33:28 2024,t3_1ftfb3j,nhl
1287,Fri Oct 4 00:02:54 2024,NaN,"Zoom in, Vegas!",Thu Oct 3 22:27:55 2024,t3_1ftfb3j,nhl
2212,Fri Oct 4 00:03:02 2024,NaN,[San Jose Sharks PR] The Sharks have added D J...,Thu Sep 19 10:38:01 2024,t3_1fk05aa,nhl
437,Fri Oct 4 00:04:04 2024,NaN,HOFer Sather retires after six decades in NHL,Wed Jun 26 12:19:52 2024,t3_1dli1s7,nhl
627,Fri Oct 4 00:04:15 2024,NaN,McDavid and Draisaitl are 1st and 2nd in point...,Sun Jun 16 09:55:55 2024,t3_1dg15bm,nhl
778,Fri Oct 4 00:04:24 2024,NaN,Panthers’ first goal on the NHL in American Si...,Sat Jun 8 18:43:18 2024,NaN,nhl
1094,Fri Oct 4 00:04:24 2024,NaN,Using Data Visualizations to compare Hurricane...,Sun Jun 9 21:34:58 2024,NaN,nhl
122,Fri Oct 4 00:04:24 2024,NaN,Bennett punch to Marchand - the smoking gun an...,Tue Jun 11 16:33:03 2024,NaN,nhl
1267,Fri Oct 4 00:04:24 2024,NaN,Why the hell did they bring the cup out?,Sat Jun 8 17:32:13 2024,NaN,nhl


In [21]:
# Comparing shape after merge
merged_pulls.shape

(2440, 6)

### Writing Pulls

In [22]:
# Checking if any null values to handle
# Some early posts names were not gathered
# Some didnt have anchors as the pull reached the bottom of the page
merged_pulls.isnull().sum()

time_pulled        0
fullname         446
titles             0
times_created      0
anchors          170
subreddit          0
dtype: int64

In [23]:
merged_pulls.to_csv('./merged_pulls.csv', index=False)

# Merged Data Scrapes
___

The data has been merged together.  The data is 2440 posts and 6 features.  The data is really just one feature waiting to be processed, the post titles.  This data was selected as it feels like it carries significant words that are particular to each class.  The number of posts should be sufficient to produce a decent model.
___